<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS12_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [2]:
### YOUR CODE STARTS HERE
# Start by importing packages to work with 
import pandas as pd 
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
# preview the file in textedit and then load into co lab
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data


--2020-01-21 20:55:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-01-21 20:55:36 (608 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [3]:
#read in the csv file
voting_data = pd.read_csv('house-votes-84.data')
voting_data.head()



,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
# Need Column headers
# Missing data
# Decimal values 


In [0]:
column_headers = ['party', 'handicapped-infants',
                  'water-project-cost-sharing', 'adoption-of-the-budget-resolution',
                  'physician-fee-freeze','el-salvador-aid', 'religious-groups-in-schools',
                  'anti-satellite-test-ban', 'aid-to-nicaraguan-contras' ,
                  'mx-missile', 'immigration', 'synfuels-corporation-cutback',
                  'education-spending', 'superfund-right-to-sue', 'crime',
                  'duty-free-exports', 'export-administration-act-south-africa']

voting_data.columns = column_headers

voting_data = voting_data.replace({'?':np.NaN, 'y':1,'n':0})
          

In [16]:
republican = voting_data[voting_data['party']== 'republican']
democrat = voting_data[voting_data['party']== 'democrat']

print(republican.shape)
print(democrat.shape)

(167, 17)
(267, 17)


In [6]:
republican.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
6,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
9,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0
10,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,NaN


In [7]:
# republicans favor education spending legestlation more than democrats 

ttest_ind(republican['education-spending'], democrat['education-spending'], nan_policy='omit')

Ttest_indResult(statistic=20.414298768685285, pvalue=4.967619782338976e-64)

In [12]:
republican['education-spending'].value_counts(dropna=False)

1.0    134
0.0     20
NaN     13
Name: education-spending, dtype: int64

In [13]:
democrat['education-spending'].value_counts(dropna=False)

0.0    213
1.0     36
NaN     18
Name: education-spending, dtype: int64

In [26]:
#democrats support mx missile more than republicans. 

ttest_ind(republican['mx-missile'], democrat['mx-missile'], nan_policy='omit')

Ttest_indResult(statistic=-16.371375965888596, pvalue=1.0255175625864558e-46)

In [27]:
republican['mx-missile'].value_counts(dropna=False)

0.0    145
1.0     19
NaN      3
Name: mx-missile, dtype: int64

In [28]:
democrat['mx-missile'].value_counts(dropna=False)

1.0    188
0.0     60
NaN     19
Name: mx-missile, dtype: int64

In [10]:
#both partys generally support water project legestlation 
ttest_ind(republican['water-project-cost-sharing'], democrat['water-project-cost-sharing'], nan_policy='omit')

Ttest_indResult(statistic=0.02491808700047811, pvalue=0.9801332440121653)